# Generate Excel report and bar chart

In [ ]:
import json
import pandas as pd

with open("data/monitored_emails.json") as fh:
    emails = json.load(fh)
with open("data/breach_information.json") as fh:
    breaches = json.load(fh)

user_info_tags = ["Email addresses", "Names", "Usernames", "Phone numbers", "Social media profiles"]
credential_tags = ["Passwords", "Auth tokens", "Password hints", "Email messages", "Survey results", "Website activity"]

combined = []
for email, data in emails.items():
    for breach in data["breaches"]:
        data = {
                "Account": email,
                "AccountName": email[:email.rfind("@")],
                "AccountDoman": email[email.rfind("@")+1:],
                "BreachTitle": breaches[breach]["Title"],
                "BreachDate": breaches[breach]["BreachDate"],
                "Domain": breaches[breach]["Domain"],
                "LeakedUserInfo": ", ".join([item for item in breaches[breach]["DataClasses"] if item in user_info_tags]),
                "LeakedCredentials": ", ".join([item for item in breaches[breach]["DataClasses"] if item in credential_tags]),
                "Verified": breaches[breach]["IsVerified"],
                "CompromisedData": ", ".join(breaches[breach]["DataClasses"]),
                "IsSpamList": breaches[breach]["IsSpamList"],
                "IsSensitive": breaches[breach]["IsSensitive"],
                "IsMalware": breaches[breach]["IsMalware"]
        }
        combined.append(data)

df = pd.DataFrame(combined)
df2 = pd.DataFrame(breaches).T

with pd.ExcelWriter("Compromised account report.xlsx") as writer: 
    df.to_excel(writer, sheet_name="CompromisedAccounts", index=False)
    fname = input("Filepath of previous report: ")
    previous = None
    try:
        previous = pd.read_excel(fname).T.to_dict()
    except:
        print("Failed to load previous file, skipping")
    if previous:
        prev_index = {f"{previous[idx]['Account']}_{previous[idx]['BreachTitle']}":idx for idx in previous}
        new_findings = []
        for acct in combined:
            search_term = f"{acct['Account']}_{acct['BreachTitle']}"
            if search_term not in prev_index:
                new_findings.append(acct)
        df_new = pd.DataFrame(new_findings)
        if not df_new.empty:
            df_new.to_excel(writer, sheet_name="Changes", index=False)
    
    df2[["Title", "Domain", "BreachDate", "PwnCount", 
        "DataClasses", "IsVerified", "IsFabricated", "IsSensitive", 
        "IsRetired", "IsSpamList", "IsMalware", "KIT_NOTIFICATION", "Description"]].to_excel(writer, sheet_name="BreachInfo", index=False)
    
sel = [False if not val else True for val in df.LeakedCredentials]

In [ ]:
for account in df[df.BreachTitle=="Internet Archive"].Account.unique().tolist():
    print(account)

In [ ]:
df[sel].groupby("BreachTitle").BreachTitle.count().sort_values().plot(kind="barh", figsize=(20,15))

In [ ]:
df[sel].groupby("BreachTitle").BreachTitle.count().sort_values().plot(kind="barh", figsize=(20,15))

In [ ]:
from app.monitor import create_table
from requests import post

ACCOUNT = input("Email account to generate notice for")
REPORTER = input("Reporter navn:", "Sikkerhetsavdelingen")
DOMAIN = input("Fresh service domain", "")
if DOMAIN:
    TICKET = input("Fresh ticket id")
    KEY = input("Fresh API key")

if ACCOUNT and ACCOUNT in emails:
    LEAKS = create_table(breaches=breaches, findings=emails["DOMAIN"]["breaches"])
    notice = f"""Hei.<br />
Din brukerkonto med epost, <b>{ACCOUNT}</b>, har blitt oppdaget i en datalekkasje og det er sterkt anmodet å bytte passord på denne brukerkontoen snarest mulig.<br />
Hvis du også har <b>gjenbrukt passordet</b> på flere tjenester må disse også skiftes da passordet kan være eksponert.<br /><br />
Epost-adressen er funnet i lekkasjer for følgende tjenester. Dette betyr at tjenesten har enten utilsiktet publisert data offentlig eller blitt kompromittert av trusselaktører, med det resultat at dine data har blitt offentligjort.<br />
Under finner du informasjonen med beskrivelser av hva som har skjedd og hva som er eksponert.<br />
{LEAKS}
<br />
For mer info eller spørsmål kan du svare på denne saken.<br />
Med vennlig hilsen {REPORTER}"""
    with open("temp_notice.html", "w") as fhtml:
        fhtml.write(notice)
    if KEY:
        request = post(f"https://{DOMAIN}/api/v2/tickets/{TICKET}/reply",
            json={"body": notice},
            headers={"authorization": f"Basic {KEY}"})
        print(request.status_code, request.content)
else:
    print(f"Email: '{ACCOUNT}' did not appear in the email list")